# Bayes Theorem
DA Probability & Statistics • Lesson 3

## Topics to Cover
- Motivating question - Together
- Bayes intuition for the 2 event case - NJ
- Updating priors - Shreyas
    - Loop through various priors to demonstrate how the posterior probabilities will update
    - What is the consequence of updating priors?
    - Controversy around priors being subjective: [Prosecutor's Fallacy](https://towardsdatascience.com/the-prosecutors-fallacy-cb0da4e9c039)
- Add a partitioning event and have the students calculate the posterior based on that - Shreyas
- Bayes proof for the generalized case - NJ
- Generalized case example - Together
    - 

## Motivating Context 🤔

Between 1996-98, Sally Clark, a British woman, lost her two infant children under similarly mysterious circumstances. The defence argued that the children had died of *SIDS* - sudden infant death syndrome. However, the prosecution claimed that the probability of children from an affluent family, like Clark's, suffering from SIDS was 1 in 73 million. They used this to *prove* that she *must* be guilty of killing her children; in 1999, Clark was, in fact, convicted of murdering them.

> Sally Clark is guilty of killing her two children **because** it is *so* unlikely for them to have died mysteriously if she were innocent.

Sounds like a pretty reasonable statement right? It's inaccurate though!

**TODO:** What do you think makes this case funky?

The prosecution argued the following:

$$
\begin{split}
P(\text{innocence | evidence}) & = P(\text{evidence | innocence}) \\
P(\text{I | E}) & = P(\text{E | I})
\end{split}
$$

As we saw above, in plain English, that comparison sounds pretty normal. Mathematically though, it's really not.

From our lesson on conditional probability, we can infer:

$$
\begin{split}
P(I | E) & = \frac{P(I \cap E)}{P(E)} \\
\implies P(I \cap E) & = P(I | E)P(E)
\end{split}
$$

**and the other way too...**

$$
\begin{split}
P(E | I) & = \frac{P(I \cap E)}{P(I)} \\
\implies P(I \cap E) & = P(E | I)P(I)
\end{split}
$$

**thus**

$$
\begin{split}
P(I | E)P(E) & = P(E | I)P(I) \\ \\
\implies P(I | E) & = \frac{P(E | I)P(I)}{P(E)} \\ \\
\implies P(I | E) & \neq P(E | I)
\end{split}
$$

**CONCLUSION:** The prosecution was wrong to convict Sally Clark in the way that it did. She was eventually re-tried and acquitted but has since passed away.

The Sally Clark case was a statistical blunder. But the mistakes made by the prosecution in this case aren't exclusive to the courtroom.

A more trivial version of the **Prosecutor's Fallacy** is saying:

>My partner hasn't texted me in 3 hours - they probably don't love me.

  - $P(\text{don't love you | late text}) \neq P(\text{late text | don't love you})$
  - The latter would understandably be very high, but the former is likely because they are either busy or are fed up of sheltering in place with you. Relax a little 😉

- Posterior Probability: The conditional probability we are trying to compute.
- Likelihood: This is the inverse of the conditional probablity we are trying to compute. If we are interested in $P(A|B)$, likelihood refers to $P(B|A)$
- Prior Probability: Probability of an event before any information on a related event. Probability of our prior beliefs.
- Marginal Probability: 

## Updating Priors

Bayes Theorem, and conditional probability in general, can be framed in multiple ways:
- $P(A|B)$ = Probability that event $A$ occurs *given* that event $B$ has occured
- $P(A|E)$ = Probability of hypothesis $A$ being true *given* certain evidence $E$
- $P(A|B)$ = Updated probability of event $A$ occurring *given* additional information from event $B$

In all these framings, we:
1. take an event $A$ that has some probabilty of ocurrring in isolation - $P(A)$ - and,
2. we update that probability to some new probability based on new information obtained from another event ocurring.

- Prior probability of an event can depend on a variety of factors. 
    - Older people, on average, have a higher likelihood of contracting COVID-19 relative to younger people
    - 